# Wildfire Prediction

This Jupyter Notebook implements various MLPs and CNNs to predict whether a landscape image represents a wildfire or not. PyTorch will be the main library used for this project. 

Image Specifications:
- Type: `.jpg`
- Dimensions: 250 x 250 pixels
- Horizontal/vertical resolution: 96 dpi
- Bit depth: 24

In [3]:
# Import libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

In [4]:
# Define transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
# Load the data
train_set = torchvision.datasets.ImageFolder('forest_fire/Training and Validation', transform)
test_set = torchvision.datasets.ImageFolder('forest_fire/Testing', transform)

In [ ]:
# Global constants
input_size = 250 * 250
classes = ['is a wildfire', 'not a wildfire']
num_classes = 2

# Eight different MLP architectures to experiment
mlp_hidden_sizes = [
    [64],
    [128, 64],
    [256, 64],
    [512, 128],
    [256, 128, 64],
    [1024, 256, 64],
    [512, 256, 128, 64],
    [1024, 512, 256, 128, 64]
]

# Nine different CNN architectures to experiment
conv_sizes = [3, 5, 7]
cnn_hidden_sizes = [
    [32],
    [32, 64],
    [32, 64, 128]
]

In [ ]:
# Create a dynamic Multi-layer Perceptron
class DynamicMLP(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(DynamicMLP, self).__init__()
        layers = [nn.Flatten()]

        # Construct MLP
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size

        # Make output layer
        layers.append(nn.Linear(prev_size, num_classes))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Create a dynamic Convolutional Neural Network
class DynamicCNN(nn.Module):
    
    def __init__(self, hidden_sizes, conv_size, num_classes):
        super(DynamicCNN, self).__init__()
        layers = []

        # Construct CNN
        curr_layer = 3 # initially RGB channels
        img_dim = 250 # image dimension
        for hidden_size in hidden_sizes:
            layers.append(nn.Conv2d(curr_layer, hidden_size, conv_size))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2))
            img_dim = img_dim // 2
            curr_layer = hidden_size

        # Output layer
        layers.append(nn.Flatten())
        layers.append(nn.Linear(img_dim * img_dim * curr_layer, num_classes))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [ ]:
# Main idea on initializing/training MLPs
for mlp_arch in mlp_hidden_sizes:
    curr_mlp = DynamicMLP(input_size, mlp_arch, num_classes)    

In [ ]:
# Main idea on initializing/training CNNs
for conv_size in conv_sizes:
    for cnn_arch in cnn_hidden_sizes:
        curr_cnn = DynamicCNN(cnn_arch, conv_size, num_classes)